In [7]:
## Importing libraries

import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import drive

from sklearn.preprocessing import StandardScaler

drive.mount('/content/drive', force_remount=True)

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


### Count - FE1

In [25]:
## Importing data

df_train = pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/test.csv')

In [27]:
## Fake vs. Real data 

test = df_test.drop(['ID_code'], axis=1).values

unique_count = np.zeros_like(test)

for feature in range(test.shape[1]):
    _, index, count = np.unique(test[:, feature], return_counts=True, return_index=True)
    unique_count[index[count == 1], feature] += 1
    
real_samples = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synth_samples = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

In [31]:
test = df_test.iloc[real_samples]
train = df_train.copy()

In [32]:

drop_vars = [7,
            10,
            17,
            27,
            29,
            30,
            38,
            41,
            46,
            96,
            100,
            103,
            126,
            158,
            185]

var_len = 200 - len(drop_vars)

# =============================================================================
# load
# =============================================================================


X_train = train.iloc[:, 2:].values
y_train = train.target.values

X_test = test.iloc[:, 1:].values

X = np.concatenate([X_train, X_test], axis=0)

reverse_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 15, 16, 18, 19, 22, 24, 25, 26,
                27, 29, 32, 35, 37, 40, 41, 47, 48, 49, 51, 52, 53, 55, 60, 61,
                62, 65, 66, 67, 69, 70, 71, 74, 78, 79, 82, 84, 89, 90, 91, 94,
                95, 96, 97, 99, 103, 105, 106, 110, 111, 112, 118, 119, 125, 128,
                130, 133, 134, 135, 137, 138, 140, 144, 145, 147, 151, 155, 157,
                159, 161, 162, 163, 164, 167, 168, 170, 171, 173, 175, 176, 179,
                180, 181, 184, 185, 187, 189, 190, 191, 195, 196, 199,
                
                ]

for j in reverse_list:
    X[:, j] *= -1

In [33]:
# drop
X = np.delete(X, drop_vars, 1)


# scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# count encoding
X_cnt = np.zeros((len(X), var_len * 4))

for j in tqdm(range(var_len)):
    for i in range(1, 4):
        x = np.round(X[:, j], i+1)
        dic = pd.value_counts(x).to_dict()
        X_cnt[:, i+j*4] = pd.Series(x).map(dic)
    x = X[:, j]
    dic = pd.value_counts(x).to_dict()
    X_cnt[:, j*4] = pd.Series(x).map(dic)

# raw + count feature
X_raw = X.copy() # rename for readable

X = np.zeros((len(X_raw), var_len * 5))
for j in tqdm(range(var_len)):
    X[:, 5*j+1:5*j+5] = X_cnt[:, 4*j:4*j+4]
    X[:, 5*j] = X_raw[:, j]

100%|██████████| 185/185 [00:05<00:00, 36.32it/s]


In [37]:
df = pd.DataFrame(X)
data = df[0:200000]
etd = df[200000:]

In [40]:
data.to_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe1-categorical/train_fe1_categorical.csv", index=False)
etd.to_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe1-categorical/test_fe1_categorical.csv", index=False)